In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adadelta

/Users/tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Agent

In [2]:
class DQN_Agent:
    #
    # Initializes attributes and constructs CNN model and target_model
    #
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=20000)
        
        # Hyperparameters
        self.gamma = 0.9            # Discount rate
        self.epsilon = 1.0          # Exploration rate
        self.epsilon_min = 0.1      # Minimal exploration rate (epsilon-greedy)
        self.epsilon_decay = 0.995  # Decay rate for epsilon
        self.update_rate = 10000    # number of steps until updating the target network
        
        # Construct DQN models
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.model.summary()

    #
    # Constructs CNN
    #
    def _build_model(self):
        model = Sequential()
        
        # Conv Layers
        model.add(Conv2D(32, (8, 8), strides=4, padding='same', input_shape=self.state_size))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (4, 4), strides=2, padding='same'))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (3, 3), strides=1, padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())

        # FC Layers
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adadelta())
        return model

    #
    # Stores experience in replay memory
    #
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    #
    # Chooses action based on epsilon-greedy policy
    #
    def act(self, state):
        # Random exploration
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        
        return np.argmax(act_values[0])  # Returns action using policy

    #
    # Trains the model using randomly selected experiences in the replay memory
    #
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            
            if not done:
                target = (reward + self.gamma * np.amax(self.target_model.predict(next_state)))
            else:
                target = reward
                
            # Construct the target vector as follows:
            # 1. Use the current model to output the Q-value predictions
            target_f = self.model.predict(state)
            
            # 2. Rewrite the chosen action value with the computed target
            target_f[0][action] = target
            
            # 3. Use vectors in the objective computation
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    #
    # Sets the target model parameters to the current model parameters
    #
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
            
    #
    # Loads a saved model
    #
    def load(self, name):
        self.model.load_weights(name)

    #
    # Saves parameters of a trained model
    #
    def save(self, name):
        self.model.save_weights(name)

# Preprocessing

In [3]:
# Helpful preprocessing taken from github.com/ageron/tiny-dqn
def process_frame(frame):
    mspacman_color = np.array([210, 164, 74]).mean()
    img = frame[1:176:2, ::2] # crop and downsize
    img = img.mean(axis=2) # to greyscale
    img[img==mspacman_color] = 0 # Improve contrast
    img = (img - 128) / 128 - 1 # normalize from -1. to 1.
    return img.reshape(88, 80, 1)

# Environment

In [4]:
env = gym.make('MsPacman-v0')
state_size = (88, 80, 1)
action_size = env.action_space.n
agent = DQN_Agent(state_size, action_size)

episodes = 500
batch_size = 16
skip_start = 90  # MsPacman-v0 waits for 90 actions before the episode begins
total_time = 0   # Counter for total number of steps taken
done = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 20, 32)        2080      
_________________________________________________________________
activation_1 (Activation)    (None, 22, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 10, 64)        32832     
_________________________________________________________________
activation_2 (Activation)    (None, 11, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 10, 64)        36928     
_________________________________________________________________
activation_3 (Activation)    (None, 11, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7040)              0         
__________

In [ ]:
for e in range(episodes):
    total_reward = 0
    state = process_frame(env.reset())
    state = np.expand_dims(state, axis=0)
    
    for skip in range(skip_start): # skip the start of each game
        env.step(0)
    
    for time in range(20000):
        total_time += 1
        
        # Every update_rate timesteps we update the target network
        if total_time % agent.update_rate == 0:
            agent.update_target_model()
        
        env.render()
        
        # Transition Dynamics
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.expand_dims(process_frame(next_state), axis=0)
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        
        if done:
            print("episode: {}/{}, reward: {}, time: {}, total transitions: {}"
                  .format(e+1, episodes, total_reward, time, total_time))
            break
            
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 1/500, reward: 370.0, time: 698, e: 0.099682
episode: 2/500, reward: 230.0, time: 597, e: 0.099682
episode: 3/500, reward: 200.0, time: 575, e: 0.099682
episode: 4/500, reward: 120.0, time: 440, e: 0.099682
episode: 5/500, reward: 250.0, time: 473, e: 0.099682
episode: 6/500, reward: 370.0, time: 775, e: 0.099682
episode: 7/500, reward: 150.0, time: 387, e: 0.099682
episode: 8/500, reward: 150.0, time: 302, e: 0.099682
episode: 9/500, reward: 330.0, time: 626, e: 0.099682
episode: 10/500, reward: 1160.0, time: 1020, e: 0.099682
episode: 11/500, reward: 380.0, time: 731, e: 0.099682
episode: 12/500, reward: 230.0, time: 520, e: 0.099682
episode: 13/500, reward: 270.0, time: 697, e: 0.099682
episode: 14/500, reward: 360.0, time: 594, e: 0.099682
episode: 15/500, reward: 320.0, time: 394, e: 0.099682
episode: 16/500, reward: 100.0, time: 349, e: 0.099682
episode: 17/500, reward: 280.0, time: 475, e: 0.099682
episode: 18/500, reward: 180.0, time: 328, e: 0.099682
episode: 19/500, 

In [ ]:
agent.save()